#### File này kiểm tra lý do tại sao nên bỏ các cột 

In [2]:
import pandas as pd
import os

# Cài đặt hiển thị đẹp
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)

# Đường dẫn thư mục chứa file
RAW_PATH = "/Users/nthanhdat/Documents/Fpt_Polytechnic/Graduation_Project/data_pipeline/raw_data"

def load_and_prepare(files, drop_airline=False, drop_is_duplicate=False):
    dfs = []
    for file in files:
        df = pd.read_csv(os.path.join(RAW_PATH, file))
        df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
        
        if drop_airline and "airline" in df.columns:
            df.drop(columns="airline", inplace=True)
        if drop_is_duplicate and "is_duplicate" in df.columns:
            df.drop(columns="is_duplicate", inplace=True)
        
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# 1. Vietjet: xoá airline & is_duplicate
df_vietjet = load_and_prepare(["vietjet_1.csv", "vietjet_2.csv"],
                              drop_airline=True, drop_is_duplicate=True)

# 2. Vietnam Airlines: xoá airline
df_vnairlines = load_and_prepare(["VNairlines_1_1.csv", "VNairlines_2_2.csv"],
                                 drop_airline=True)

# 3. Bamboo: giữ nguyên hoàn toàn
df_bamboo = load_and_prepare(["BambooAirway.csv"])

# Gộp tất cả
df_all = pd.concat([df_vietjet, df_vnairlines, df_bamboo], ignore_index=True)

# Xem kết quả
print("Tổng số dòng sau khi gộp:", len(df_all))
print(df_all.head(10))


Tổng số dòng sau khi gộp: 2255788
                     source.name        date    route                        flight_id origin destination fare_class fare_code     price_vnd          taxes departure_time arrival_time booking_class  \
0  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ568-DLIDAD-2025-1-01-1450    DLI         DAD        EL1   QPXVNWS  1.860040e+06  572426.357322            NaN          NaN             Z   
1  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ886-DLIDAD-2025-1-01-1450    DLI         DAD        EC1     ZVNWP  1.929579e+06  577492.077298            NaN          NaN             Z   
2  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ430-DLIDAD-2025-1-01-1450    DLI         DAD        EF1     ZVNWP  2.148786e+06  594070.797220            NaN          NaN             Z   
3  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ180-DLIDAD-2025-1-01-1450    DLI         DAD        DC1     ZVNWP  2.222009e+06  599136.517197       

In [5]:
# Status bỏ vì chứa soure file thôi không có thông tin gì khác
# Cột flight_id giữ
flight_id_counts = df_all['flight_id'].value_counts()
print(flight_id_counts.head(10))  # Xem top 10 chuyến bị trùng

df_dupes = df_all[df_all['flight_id'] == "SEG-VN1341-CXRSGN-2025-06-14-0840"]

print("✅ Số bản ghi:", len(df_dupes))
print("✅ Số mức giá khác nhau:", df_dupes['price_vnd'].nunique())
print("🪙 Các mức giá:")
print(df_dupes['price_vnd'].unique())





flight_id
SEG-VN1341-CXRSGN-2025-06-14-0840    198
SEG-VN1341-CXRSGN-2025-06-28-0840    196
SEG-VN1341-CXRSGN-2025-06-26-0840    189
SEG-VN1341-CXRSGN-2025-06-07-0840    180
SEG-VN1341-CXRSGN-2025-09-22-0840    175
SEG-VN1341-CXRSGN-2025-06-12-0840    175
SEG-VN1341-CXRSGN-2025-08-07-0840    175
SEG-VN1341-CXRSGN-2025-10-13-0840    175
SEG-VN1341-CXRSGN-2025-08-27-0840    175
SEG-VN1341-CXRSGN-2025-10-08-0840    168
Name: count, dtype: int64
✅ Số bản ghi: 198
✅ Số mức giá khác nhau: 39
🪙 Các mức giá:
[ 3523000.  4228000.  4501000.  5365000.  7065000.  8253000.  3674000.
  7497000.  3996000.  4280000.  5473000.  5797000.  7929000.  8685000.
  3631000.  4225000.  5362000.  5851000.  6715000. 11925000. 12681000.
  6175000.  7106000. 11493000.  3836000.  9009000.  4020000.  3569000.
  3681000.  3907000.  4555000.  6741000.  6417000.  1616000.  1900000.
  2013000.  2337000.  3268000.  4024000.]


In [8]:
#### Cột departure_time arrival_time bỏ
# Kiểm tra tổng số dòng
total_rows = len(df_all)

# Đếm số lượng NaN trong từng cột
departure_missing = df_all['departure_time'].isna().sum()
arrival_missing = df_all['arrival_time'].isna().sum()

# In kết quả
print(f"📌 Tổng số dòng: {total_rows}")
print(f"🛫 'departure_time' bị thiếu: {departure_missing} dòng ({departure_missing / total_rows:.2%})")
print(f"🛬 'arrival_time' bị thiếu:   {arrival_missing} dòng ({arrival_missing / total_rows:.2%})")

has_departure_data = df_all['departure_time'].notna().any()
has_arrival_data = df_all['arrival_time'].notna().any()

print("✅ Có dữ liệu departure_time không?:", has_departure_data)
print("✅ Có dữ liệu arrival_time không?:", has_arrival_data)

 

📌 Tổng số dòng: 2255788
🛫 'departure_time' bị thiếu: 2231998 dòng (98.95%)
🛬 'arrival_time' bị thiếu:   2231998 dòng (98.95%)
✅ Có dữ liệu departure_time không?: True
✅ Có dữ liệu arrival_time không?: True


In [ ]:
# Đoạn code kiểm tra 5 cột: quota, status, service_code, fare_conditions, operating_airline_code có nên xóa không ?
# Kết quả status là xóa 
columns_to_check = ['quota', 'status', 'service_code', 'fare_conditions', 'operating_airline_code']

total_rows = len(df_all)

for col in columns_to_check:
    print(f"\n🔍 Cột: {col}")
    if col not in df_all.columns:
        print("❗ Cột không tồn tại trong dữ liệu.")
        continue

    unique_vals = df_all[col].dropna().unique()
    unique_count = len(unique_vals)
    null_count = df_all[col].isna().sum()
    null_percent = null_count / total_rows * 100

    print(f"• Số giá trị khác nhau (unique): {unique_count}")
    print(f"• Số dòng bị thiếu (NaN): {null_count} / {total_rows} ({null_percent:.2f}%)")
    print(f"• Một vài giá trị đầu tiên: {unique_vals[:5]}")

    # Gợi ý giữ/xoá đơn giản
    if unique_count == 1 and null_percent < 10:
        print("👉 Gợi ý: ❌ Có thể XÓA (chỉ có 1 giá trị)")
    elif null_percent > 95:
        print("👉 Gợi ý: ❌ Có thể XÓA (quá nhiều dữ liệu thiếu)")
    else:
        print("👉 Gợi ý: ✅ CÂN NHẮC GIỮ (có thông tin phân biệt)")



🔍 Cột: quota
• Số giá trị khác nhau (unique): 9
• Số dòng bị thiếu (NaN): 383 / 2255788 (0.02%)
• Một vài giá trị đầu tiên: [2. 8. 9. 4. 5.]
👉 Gợi ý: ✅ CÂN NHẮC GIỮ (có thông tin phân biệt)

🔍 Cột: status
• Số giá trị khác nhau (unique): 1
• Số dòng bị thiếu (NaN): 383 / 2255788 (0.02%)
• Một vài giá trị đầu tiên: ['HK']
👉 Gợi ý: ❌ Có thể XÓA (chỉ có 1 giá trị)

🔍 Cột: service_code
• Số giá trị khác nhau (unique): 21
• Số dòng bị thiếu (NaN): 24173 / 2255788 (1.07%)
• Một vài giá trị đầu tiên: ['FBA4-C0FMC2' 'FBA3-C0GOC1' 'FBA5-C0FMC2' 'FBA1-C0GOC1' 'FBA2-C0FMC2']
👉 Gợi ý: ✅ CÂN NHẮC GIỮ (có thông tin phân biệt)

🔍 Cột: fare_conditions
• Số giá trị khác nhau (unique): 22
• Số dòng bị thiếu (NaN): 383 / 2255788 (0.02%)
• Một vài giá trị đầu tiên: ['PEN19,PEN20,PEN21' 'PEN16,PEN17,PEN18' 'PEN22,PEN23,PEN24'
 'PEN28,PEN29,PEN30' 'PEN43,PEN44,PEN45']
👉 Gợi ý: ✅ CÂN NHẮC GIỮ (có thông tin phân biệt)

🔍 Cột: operating_airline_code
• Số giá trị khác nhau (unique): 3
• Số dòng bị thiếu (NaN):